# IMPORTS # 

In [ ]:
import json
import gc
import torchvision.transforms.functional as F
import torchvision.transforms as T
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import json
import os
import cv2
import numpy as np
from tqdm import tqdm 
from pathlib import Path
import torch
import cv2
import numpy as np
import json
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil
from torchvision.models.detection.rpn import AnchorGenerator,RPNHead
from tqdm.notebook import tqdm
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Studio di xView 

In [ ]:
geojson_path = '/kaggle/input/xview-dataset/train_labels/xView_train.geojson' 
with open(geojson_path) as f:
    data = json.load(f)

class_ids = [feature['properties']['type_id'] for feature in data['features']]

df_classes = pd.DataFrame(class_ids, columns=['type_id'])

counts = df_classes['type_id'].value_counts().sort_index()

num_classes = df_classes['type_id'].nunique()

print(f"Totale classi: {num_classes}")
print(f"Totale oggetti annotati: {len(df_classes)}")
print("\n--- Conteggio per Class ID ---")
print(df_classes['type_id'].value_counts().sort_index(ascending=False).head(60)) #altrimenti me ne lista solamente 10 (5 top e 5 bottom)

# Tiling di xView # 

# Funzione per prelevare immagini xview #

trasformo il geojson in un json normale (per ogni img)

In [ ]:
def load_xview_data(geojson_path): # prendo geojson e creo un json/dizionario organizzato per id dell'img

    print("Caricamento GeoJSON... (può richiedere un po')")
    with open(geojson_path) as f:
        data = json.load(f)
    
    coords_by_img = {}
    
    for feature in tqdm(data['features']):
        props = feature['properties']
        img_name = props['image_id']
        bounds = [int(float(x)) for x in props['bounds_imcoords'].split(',')]
        type_id = props['type_id']
        
        if img_name not in coords_by_img:
            coords_by_img[img_name] = [] # dizionario di liste (key: img_name -> value: lista di annotazioni)
            
        coords_by_img[img_name].append({
            'bounds': bounds,
            'type_id': type_id
        })
        
    return coords_by_img

# Funzione di Tiling #
Usata per entrambi i dataset (train,val)

In [ ]:
def tile_dataset(image_list, output_dir, output_json_path, TILE_SIZE, STRIDE, coords_by_img):
    subset_data = []
    
    # --- PARAMETRI DI FILTRAGGIO ---
    MIN_AREA = 16 * 16  # 256 pixel quadrati (es. un box 16x16)
    MAX_TRUNCATION = 0.50 # Scarta se più del 50% dell'oggetto è fuori dal tile
    MIN_DIM = 4 # Lato minimo in pixel
    # -------------------------------

    for img_name in tqdm(image_list):
        img_path = os.path.join(INPUT_IMAGES_DIR, img_name)
        img = cv2.imread(img_path)
        
        if img is None: 
            continue
                
        h, w, _ = img.shape
        original_boxes = coords_by_img[img_name]

        # Tiling Loop
        for y in range(0, h, STRIDE):
            for x in range(0, w, STRIDE):
                x2 = min(x + TILE_SIZE, w)
                y2 = min(y + TILE_SIZE, h)
                x1 = x2 - TILE_SIZE
                y1 = y2 - TILE_SIZE
                if x1 < 0: x1 = 0 
                if y1 < 0: y1 = 0
                
                # Estrai la tile
                tile_img = img[y1:y2, x1:x2]
                
                valid_objects = []
                for obj in original_boxes:
                    bx1, by1, bx2, by2 = obj['bounds']
                    
                    # Calcolo Area Originale (per il troncamento)
                    original_w = bx2 - bx1
                    original_h = by2 - by1
                    original_area = original_w * original_h
                    
                    # Intersezione (Box nel Tile)
                    ix1 = max(x1, bx1)
                    iy1 = max(y1, by1)
                    ix2 = min(x2, bx2) 
                    iy2 = min(y2, by2)
                    
                    if ix1 < ix2 and iy1 < iy2:
                        # Coordinate locali nella tile
                        nx1 = ix1 - x1
                        ny1 = iy1 - y1
                        nx2 = ix2 - x1
                        ny2 = iy2 - y1
                        
                        new_w = nx2 - nx1
                        new_h = ny2 - ny1
                        new_area = new_w * new_h
                        
                        # --- FILTRI AVANZATI ---
                        
                        # 1. Filtro Dimensione Minima (Pixel)
                        if new_w < MIN_DIM or new_h < MIN_DIM:
                            continue # Troppo sottile/piccolo
                            
                        # 2. Filtro Area Minima
                        if new_area < MIN_AREA:
                            continue # Oggetto troppo piccolo per essere riconosciuto
                            
                        # 3. Filtro Troncamento (Cruciale!)
                        # Quanta parte dell'oggetto originale è finita in questo tile?
                        # Se ho solo il 30% dell'oggetto (truncation > 0.7), lo butto.
                        # (1 - ratio) ci dà la frazione mancante.
                        visible_ratio = new_area / original_area
                        truncation = 1.0 - visible_ratio
                        
                        if truncation > MAX_TRUNCATION:
                            continue # Oggetto troppo tagliato
                        
                        # Se passa i filtri, lo salviamo
                        valid_objects.append({
                            'bounds': [nx1, ny1, nx2, ny2],
                            'type_id': obj['type_id']
                        })
                
                # Salva solo se la tile ha oggetti validi sopravvissuti ai filtri
                if len(valid_objects) > 0:
                    tile_filename = f"{Path(img_name).stem}_tile_{x}_{y}.jpg"
                    save_path = os.path.join(output_dir, tile_filename)
                    cv2.imwrite(save_path, tile_img)
                    subset_data.append({
                        'img_name': tile_filename,
                        'objects': valid_objects
                    })

    print(f"Creati {len(subset_data)} tiles validi.")
                    
    with open(output_json_path, 'w') as f:
        json.dump(subset_data, f)

In [ ]:
# --- CONFIGURAZIONE ---
INPUT_IMAGES_DIR = '/kaggle/input/xview-dataset/train_images/train_images' 
INPUT_GEOJSON = '/kaggle/input/xview-dataset/train_labels/xView_train.geojson'

# Cartelle di output
TRAIN_DIR = '/kaggle/working/train_tiles'
VAL_DIR = '/kaggle/working/val_tiles'
TRAIN_JSON = '/kaggle/working/train_annotations.json'
VAL_JSON = '/kaggle/working/val_annotations.json'
TEST_JSON = '/kaggle/working/test_annotations.json'
TEST_GT_JSON = '/kaggle/working/test_gt_annotations.json'

# Pulizia cartelle
if os.path.exists(TRAIN_DIR): shutil.rmtree(TRAIN_DIR)
if os.path.exists(VAL_DIR): shutil.rmtree(VAL_DIR)
os.makedirs(TRAIN_DIR)
os.makedirs(VAL_DIR)

TILE_SIZE = 512
STRIDE = 400

def create_datasets(num_train, num_val, num_test):
    
    # 1. Carica TUTTE le immagini disponibili
    print("Caricamento lista immagini...")
    coords_by_img = load_xview_data(INPUT_GEOJSON) 
    
    # Questa è l'urna con tutte le 847 immagini
    available_images = list(coords_by_img.keys()) 
    total_available = len(available_images)
    
    print(f"Totale immagini nell'urna: {total_available}")
    
    # Controllo sicurezza
    if (num_train + num_val + num_test) > total_available:
        print("ERRORE: Hai chiesto più immagini di quelle disponibili!")
        return

    # Liste vuote da riempire
    train_imgs = []
    val_imgs = []
    test_imgs = []

    # --- ESTRAZIONE TRAINING SET ---
    print(f"\n--- Estrazione casuale di {num_train} immagini per il TRAIN ---")
    random.seed(42) # Seed per riproducibilità (rimuovilo se vuoi casualità pura ogni volta)
    
    for i in range(num_train):
        # 1. Genera un indice casuale tra 0 e l'ultimo indice disponibile rimasto
        # Nota: len(available_images) diminuisce a ogni giro!
        random_index = random.randint(0, len(available_images) - 1)
        
        # 2. 'pop' estrae l'elemento a quell'indice e lo rimuove dalla lista originale
        selected_img = available_images.pop(random_index)
        
        # 3. Aggiungilo alla lista train
        train_imgs.append(selected_img)

    # --- ESTRAZIONE VALIDATION SET ---
    print(f"--- Estrazione casuale di {num_val} immagini per il VALIDATION ---")
    for i in range(num_val):
        # Pesco dalle rimanenti
        random_index = random.randint(0, len(available_images) - 1)
        selected_img = available_images.pop(random_index)
        val_imgs.append(selected_img)

    # --- ESTRAZIONE TEST SET ---
    print(f"--- Estrazione casuale di {num_test} immagini per il TEST ---")
    for i in range(num_test):
        # Pesco dalle rimanenti
        random_index = random.randint(0, len(available_images) - 1)
        selected_img = available_images.pop(random_index)
        test_imgs.append(selected_img)
        
    # --- RIEPILOGO ---
    print(f"\nRiepilogo Assegnazioni:")
    print(f"TRAIN: {len(train_imgs)} immagini")
    print(f"VAL:   {len(val_imgs)} immagini")
    print(f"TEST:  {len(test_imgs)} immagini")
    print(f"RIMASTE (non usate): {len(available_images)} immagini")

    # --- GENERAZIONE DEI FILE ---
    print("\n--- Generazione Tiles Training ---")
    tile_dataset(train_imgs, TRAIN_DIR, TRAIN_JSON, TILE_SIZE, STRIDE, coords_by_img)
    
    print("--- Generazione Tiles Validation ---")
    tile_dataset(val_imgs, VAL_DIR, VAL_JSON, TILE_SIZE, STRIDE, coords_by_img)

    print("--- Generazione Test Set (JSON) ---")
    with open(TEST_JSON, 'w') as f:
        json.dump(test_imgs, f)
    
    test_ground_truth = {img: coords_by_img[img] for img in test_imgs}
    with open(TEST_GT_JSON, 'w') as f:
        json.dump(test_ground_truth, f)


create_datasets(num_train=700, num_val=100, num_test=47)

# Modifica di Compose e ToTensor #
Modifico le seguenti classi per consentire il compose (e le transform) di img + annotations
prima avevo infatti che Compose accettava solo 1 param. : l'img, ora devo flippare anche le annotations

In [ ]:
# 1. Classe Compose personalizzata che accetta (image, target)
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

# 2. ToTensor che accetta (image, target)
class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

# 3. Flip Orizzontale (Immagine + Box)
class RandomHorizontalFlip(object):
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            
            # Flippo l'immagine
            image = image.flip(-1)
            
            # Flippo i box
            bbox = target["boxes"]
            # x1_new = width - x2_old
            # x2_new = width - x1_old
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
        return image, target

# 4. Flip Verticale (Immagine + Box) - Fondamentale per il satellite
class RandomVerticalFlip(object):
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            
            # Flippo l'immagine
            image = image.flip(-2)
            
            # Flippo i box
            bbox = target["boxes"]
            # y1_new = height - y2_old
            # y2_new = height - y1_old
            bbox[:, [1, 3]] = height - bbox[:, [3, 1]]
            target["boxes"] = bbox
        return image, target

# classe color Jitter per uniformare contrasto
class ColorJitter(object):
    def __init__(self, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05):
        # Usiamo l'implementazione veloce di PyTorch internamente
        self.transform = T.ColorJitter(brightness=brightness, 
                                       contrast=contrast, 
                                       saturation=saturation, 
                                       hue=hue)

    def __call__(self, image, target):
        # Applico il jitter SOLO all'immagine
        image = self.transform(image)
        
        # Restituisco l'immagine modificata e il target (box) INALTERATO
        return image, target



# Classe Dataset #

In [ ]:
class MioxViewDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        
        # Carico il file delle annotazioni
        with open(annotation_file, 'r') as f:
            self.imgs_data = json.load(f)

        # Mapping delle classi
        self.class_map = {11: 1, 12: 2, 13: 3, 15: 4, 17: 5, 18: 6, 19: 7, 20: 8,
            21: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14, 28: 15, 29: 16,
            32: 17, 33: 18, 34: 19, 35: 20, 36: 21, 37: 22, 38: 23, 40: 24,
            41: 25, 42: 26, 44: 27, 45: 28, 47: 29, 49: 30, 50: 31, 51: 32,
            52: 33, 53: 34, 54: 35, 55: 36, 56: 37, 57: 38, 59: 39, 60: 40,
            61: 41, 62: 42, 63: 43, 64: 44, 65: 45, 66: 46, 71: 47, 72: 48,
            73: 49, 74: 50, 76: 51, 77: 52, 79: 53, 83: 54, 84: 55, 86: 56,
            89: 57, 91: 58, 93: 59, 94: 60}

    def __len__(self):
        return len(self.imgs_data)

    def __getitem__(self, idx):
        data = self.imgs_data[idx]
        img_path = os.path.join(self.root_dir, data['img_name'])
        
        # Caricamento immagine
        img = Image.open(img_path).convert("RGB")
        
        boxes = []
        labels = []
        
        # --- COSTANTI PER FILTRAGGIO ---
        MIN_AREA = 16 * 16  # 256 pixel quadrati
        MIN_DIM = 4         # Almeno 4px di lato
        # -------------------------------

        for obj in data['objects']:
            xmin, ymin, xmax, ymax = obj['bounds']
            
            # Calcolo dimensioni
            width = xmax - xmin
            height = ymax - ymin
            area = width * height

            # 1. Filtro Validità base
            if xmax <= xmin or ymax <= ymin:
                continue

            # 2. Filtro Dimensioni Minime (< 4px)
            # Mask R-CNN crasha se i box sono degeneri o troppo piccoli per le ancore
            if width < MIN_DIM or height < MIN_DIM:
                continue

            # 3. Filtro Area Minima (< 16x16) e Troncamento indiretto
            # Se un oggetto è troncato pesantemente, la sua area crollerà.
            # Questo check rimuove i frammenti piccoli ai bordi.
            if area < MIN_AREA:
                continue

            # 4. Check Label (Rimuovi classe 0 o non mappata)
            original_id = obj['type_id']
            label_id = self.class_map.get(original_id, 0) 
            
            if label_id == 0:
                continue

            # Se passa tutti i test, aggiungi
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label_id)

        # Conversione in Tensori
        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.tensor([idx])
        
        # Area e Iscrowd (Standard COCO)
        if len(boxes) > 0:
            target["area"] = (target["boxes"][:, 3] - target["boxes"][:, 1]) * (target["boxes"][:, 2] - target["boxes"][:, 0])
        else:
            target["area"] = torch.zeros((0,), dtype=torch.float32)
            
        target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)

        # Gestione TILE VUOTE 
        # (Se dopo il filtraggio rigoroso 16x16 non è rimasto nulla)
        if len(boxes) == 0:
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
            target["area"] = torch.zeros((0,), dtype=torch.float32)

        if self.transforms:
            img, target = self.transforms(img, target)
        else:
            img = F.to_tensor(img)

        return img, target

# Collate function rimane uguale
def collate_fn(batch):
    return tuple(zip(*batch))

# Modello #

Modifico la resnet per fare transfer learning

In [ ]:
def get_maskrcnn_model(num_classes):

    model = torchvision.models.detection.maskrcnn_resnet50_fpn(
            weights="DEFAULT",
            min_size=512, # fisso la dimensione del resize automatico della rete (default 1333x800), altrimenti la rete effettua train su dimensioni diverse che dal test
            max_size=512, # analogo
            rpn_fg_iou_thresh=0.4, # considera roi se > 0.4
            rpn_bg_iou_thresh=0.2, # considera sfondo se < 0.2
        
            # aumento le proposte per essere sicuro di non perderne
            rpn_pre_nms_top_n_train=2000, 
            rpn_post_nms_top_n_train=2000,
            # ----------------------------------
        )    

    #le ancore della rete addestrata su COCO sono troppo grandi (32x32) rispetto a quelle che servono per xView -> inserisco anche 8x8 e 16x16 
    
    # Definiamo le tue ancore personalizzate per xView
    
    anchor_sizes = ((8,), (16,), (32,), (64,), (128,))
    aspect_ratios = ((0.5, 1.0, 2.0),) * len(anchor_sizes)
    
    # Creiamo il nuovo generatore
    rpn_anchor_generator = AnchorGenerator(
        sizes=anchor_sizes, 
        aspect_ratios=aspect_ratios
    )
    
    model.rpn.anchor_generator = rpn_anchor_generator

    # SOSTITUZIONE  DELLA TESTA DELLA RPN 
    # Devo ricreare la "head" perché ora deve predire 15 ancore invece di 3.
    # Calcoliamo i canali di input (normalmente 256 per ResNet50-FPN)
    in_channels = model.rpn.head.cls_logits.in_channels
    
    # Calcoliamo quante ancore ci sono per ogni pixel (5 sizes * 3 ratios = 15)
    num_anchors = rpn_anchor_generator.num_anchors_per_location()[0]
    
    # Creiamo una nuova testa RPN fresca e pulita
    model.rpn.head = RPNHead(in_channels, num_anchors)
    
    # 2. CAMBIA LA TESTA DEI BOX (Rettangoli + Classificazione)
    in_features_box = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features_box, num_classes)

    # 3. CAMBIA LA TESTA DELLE MASCHERE (Segmentation), non ci serve a nulla -> None
    model.roi_heads.mask_predictor = None # fondamentale
    
    return model

# Transforms per data augmentation #

Altrimenti il modello funziona male (prove precedenti)

In [ ]:
def get_train_transform():
    return Compose([
        ToTensor(),                 # 1. Img -> Tensore
        
        # 2. Data Augmentation Colore (Solo Train)
        # Valori consigliati per satellite:
        # - Brightness/Contrast: 0.2-0.3 (gestisce ombre e luci forti)
        # - Hue: basso (0.05) perché un prato verde non diventa mai viola
        ColorJitter(),
        # 3. Data Augmentation Geometrica
        RandomHorizontalFlip(0.5),  
        RandomVerticalFlip(0.5)     
    ])

def get_val_transform():
    return Compose([
        ToTensor(), # In validazione niente Jitter né Flip!
    ])

In [ ]:
# 1. Dataset e Loader di TRAINING
train_dataset = MioxViewDataset(
    root_dir='/kaggle/working/train_tiles',
    annotation_file='/kaggle/working/train_annotations.json',
    transforms=get_train_transform()
)
train_loader = DataLoader(
    train_dataset, batch_size=8, shuffle=True, 
    num_workers=2, collate_fn= collate_fn
)

# 2. Dataset e Loader di VALIDATION
val_dataset = MioxViewDataset(
    root_dir='/kaggle/working/val_tiles',
    annotation_file='/kaggle/working/val_annotations.json',
    transforms=get_val_transform()
)
val_loader = DataLoader(
    val_dataset, batch_size=8, shuffle=False, 
    num_workers=2, collate_fn=collate_fn
)

#  Check delle tile dal train_loader #

In [ ]:
# Prendi un batch dal loader
images, targets = next(iter(train_loader))

# Prendi la prima immagine
img = images[0]
target = targets[0]

img = img.permute(1, 2, 0).numpy() # C,H,W -> H,W,C
img = np.clip(img, 0, 1)

plt.figure(figsize=(10, 10))
plt.imshow(img)
ax = plt.gca()

for box in target['boxes']:
    x1, y1, x2, y2 = box.numpy()
    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2)
    ax.add_patch(rect)

plt.title(f"Check Training Data: {len(target['boxes'])} boxes")
plt.show()

# Check classi nel val_dataset #

In [ ]:
TRAIN_ID_TO_NAME = {
    1: 'Fixed-wing Aircraft', 2: 'Small Aircraft', 3: 'Cargo Plane', 4: 'Helicopter',
    5: 'Passenger Vehicle', 6: 'Small Car', 7: 'Bus', 8: 'Pickup Truck',
    9: 'Utility Truck', 10: 'Truck', 11: 'Cargo Truck', 12: 'Truck w/Box',
    13: 'Truck Tractor', 14: 'Trailer', 15: 'Truck w/Flatbed', 16: 'Truck w/Liquid',
    17: 'Crane Truck', 18: 'Railway Vehicle', 19: 'Passenger Car', 20: 'Cargo Car',
    21: 'Flat Car', 22: 'Tank Car', 23: 'Locomotive', 24: 'Maritime Vessel',
    25: 'Motorboat', 26: 'Sailboat', 27: 'Tugboat', 28: 'Barge',
    29: 'Fishing Vessel', 30: 'Ferry', 31: 'Yacht', 32: 'Container Ship',
    33: 'Oil Tanker', 34: 'Engineering Vehicle', 35: 'Tower crane', 36: 'Container Crane',
    37: 'Reach Stacker', 38: 'Straddle Carrier', 39: 'Mobile Crane', 40: 'Dump Truck',
    41: 'Haul Truck', 42: 'Scraper/Tractor', 43: 'Front loader/Bulldozer', 44: 'Excavator',
    45: 'Cement Mixer', 46: 'Ground Grader', 47: 'Hut/Tent', 48: 'Shed',
    49: 'Building', 50: 'Aircraft Hangar', 51: 'Damaged Building', 52: 'Facility',
    53: 'Construction Site', 54: 'Vehicle Lot', 55: 'Helipad', 56: 'Storage Tank', 57: 'Shipping Container Lot',
    58: 'Shipping container', 59: 'Pylon', 60: 'Tower'
}

# Conta le classi nel validation set
val_class_counts = {}
for idx in range(len(val_dataset)):
    _, target = val_dataset[idx]
    labels = target['labels'].numpy()
    for l in labels:
        val_class_counts[l] = val_class_counts.get(l, 0) + 1

print("Classi presenti nel Validation Set:")
for class_id, count in sorted(val_class_counts.items()):
    name = TRAIN_ID_TO_NAME.get(class_id, str(class_id))
    print(f"{name}: {count}")

# Training rete #

In [ ]:
# --- CONFIGURAZIONE INIZIALE ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Caricamento Modello (Assicurati che get_maskrcnn_model abbia mask_predictor=None)
model = get_maskrcnn_model(num_classes=61) 
model.to(device)


# 2. Congelamento Pesi
for param in model.parameters():
    param.requires_grad = True

# Blocca C1 (Stem iniziale)
for param in model.backbone.body.conv1.parameters():
    param.requires_grad = False
for param in model.backbone.body.bn1.parameters():
    param.requires_grad = False

# Blocca C2 (Layer 1)
for param in model.backbone.body.layer1.parameters():
    param.requires_grad = False

# 3. Setup Optimizer e Scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
# 4. Setup Metrica MAP
# class_metrics=False ci dà la media totale. Se metti True, ti dà il mAP per ogni singola classe (molto verboso)
metric = MeanAveragePrecision(iou_type="bbox", class_metrics=True).to(device)

# Storico
history = {
    'train_loss': [],
    'val_loss': [],
    'train_box_loss': [],
    'val_map_50': [], # Aggiungiamo lo storico del mAP
    'val_recall': []
}

num_epochs = 25

# --- MAPPA ID -> NOME (Necessaria per la stampa dei log) ---
# Questo dizionario mappa l'ID sequenziale del modello (1-60) al Nome Reale


print("Inizio Training...")

for epoch in range(num_epochs):
    # ==========================
    # 1. FASE DI TRAINING
    # ==========================
    model.train()
    running_loss = 0.0
    running_box_loss = 0.0
    
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    
    for images, targets in train_loop:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass (in train mode restituisce le loss)
        loss_dict = model(images, targets)

        # Somma delle loss (nota: loss_mask non esiste più, quindi non serve azzerarla)
        losses = sum(loss for loss in loss_dict.values())
        
        # Monitoraggio specifico box loss
        box_loss = loss_dict['loss_box_reg']

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        # Accumulo
        running_loss += losses.item()
        running_box_loss += box_loss.item()
        
        # Aggiorno la barra di caricamento con la loss attuale
        train_loop.set_postfix(loss=losses.item())

    # Medie Training
    avg_train_loss = running_loss / len(train_loader)
    avg_box_loss = running_box_loss / len(train_loader)
    
    history['train_loss'].append(avg_train_loss)
    history['train_box_loss'].append(avg_box_loss)

    # ==========================
    # 2. FASE DI VALIDATION (Loss)
    # ==========================
    # TRUCCO: Per avere la LOSS, dobbiamo restare in model.train()
    # ma usiamo torch.no_grad() per non addestrare.
    # Questo perché model.eval() NON restituisce la loss in PyTorch Detection.
    val_running_loss = 0.0
    
    # Usiamo solo una parte del validation set per la loss se è troppo lento,
    # altrimenti usalo tutto. Qui lo uso tutto.
    with torch.no_grad():
        for images, targets in val_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            val_running_loss += losses.item()
            
    avg_val_loss = val_running_loss / len(val_loader)
    history['val_loss'].append(avg_val_loss)

    # ==========================
    # 3. FASE DI VALIDATION (Metriche: mAP, Recall)
    # ==========================
    model.eval() # ORA passiamo in modalità valutazione per le predizioni
    
    with torch.no_grad():
        for images, targets in tqdm(val_loader, desc=f"Epoch {epoch+1} [Eval]"):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            # Qui il modello restituisce PREDIZIONI (boxes, scores, labels)
            predictions = model(images)
            
            # Aggiorna la metrica accumulando i dati
            metric.update(predictions, targets)

    # Calcolo finale delle metriche per questa epoca
    # ... dopo metric.update(predictions, targets) nel loop di eval ...

    # Calcolo finale delle metriche
    val_metrics = metric.compute()
    
    # Metriche Globali
    map_50 = val_metrics['map_50'].item()
    recall = val_metrics['mar_100'].item()
    
    # --- NUOVO: GESTIONE PER CLASSE ---
    # map_per_class è un tensore. Lo convertiamo in lista CPU.
    map_per_class = val_metrics['map_per_class'] 
    
    # 1. Stampa Globale
    print(f"\n EPOCA {epoch+1}/{num_epochs}")
    print(f"   mAP @ 50 (Global): {map_50:.4f}")
    print(f"   Recall (Global):   {recall:.4f}")
    print("-" * 40)
    print(" DETTAGLIO CLASSI (Top & Flop):")

    # 2. Creiamo una lista ordinata per vedere chi va bene e chi male
    class_results = []
    
    # Iteriamo sul tensore. Nota: l'indice del tensore corrisponde al label ID.
    # Attenzione: Faster R-CNN usa 0 come background. I tuoi ID partono da 1.
    # Torchmetrics potrebbe restituire un array dove l'indice i corrisponde alla classe i.
    for i, score in enumerate(map_per_class):
        score_val = score.item()
        
        # Ignora classi con score -1 (significa che non c'erano nel validation set)
        if score_val == -1: continue
        
        # Recupera il nome
        # Se i tuoi label partono da 1, l'indice 0 del tensore potrebbe essere vuoto o background
        class_name = TRAIN_ID_TO_NAME.get(i, f"Class {i}") 
        
        class_results.append((class_name, score_val))

    # Ordiniamo dal peggiore al migliore per visualizzare
    class_results.sort(key=lambda x: x[1], reverse=True) # Dall'alto al basso

    # Stampiamo le 5 migliori e le 5 peggiori per non intasare il log
    for name, score in class_results[:5]:
        print(f"{name:<20}: {score:.4f}")
    
    if len(class_results) > 10:
        print("      ...")
        for name, score in class_results[-5:]:
             print(f"{name:<20}: {score:.4f}")
    else:
        # Se sono poche stampale tutte
        for name, score in class_results[5:]:
             print(f"      {name:<20}: {score:.4f}")

    # 3. Salvataggio nello storico (Solo globali per non fare un JSON gigante)
    history['val_map_50'].append(map_50)
    history['val_recall'].append(recall)
    
    # Se vuoi salvare tutto il dettaglio in un file separato per questa epoca:
    # with open(f'/kaggle/working/metrics_epoch_{epoch+1}.txt', 'w') as f:
    #     for name, score in class_results:
    #         f.write(f"{name}: {score}\n")

    # Reset
    metric.reset()

    # ==========================
    # 4. REPORT E SALVATAGGIO
    # ==========================
    lr_scheduler.step(avg_val_loss)
    
    print(f"\n EPOCA {epoch+1}/{num_epochs}")
    print(f"   Train Loss: {avg_train_loss:.4f} | Box Loss: {avg_box_loss:.4f}")
    print(f"   Val Loss:   {avg_val_loss:.4f}")
    print(f"   mAP @ 50:   {map_50:.4f} (Obiettivo: > 0.5)")
    print(f"   Recall:     {recall:.4f}")
    print("-" * 40)
    
    # Salva checkpoint
    torch.save(model.state_dict(), f'/kaggle/working/model_epoch_{epoch+1}.pth')
    
    # Salva history JSON aggiornata
    with open('/kaggle/working/history.json', 'w') as f:
        json.dump(history, f)

    # Pulizia Memoria
    del images, targets, predictions, loss_dict, losses
    gc.collect()
    torch.cuda.empty_cache()

    current_lr = optimizer.param_groups[0]['lr']
    print(f"   Val Loss: {avg_val_loss:.4f} | Current LR: {current_lr:.2e}")

print("Training completato!")

# Visualizzazione Grafici #

def plot_metrics(history):
    epochs = range(1, len(history['train_loss']) + 1)
    
    plt.figure(figsize=(15, 6))

    # --- GRAFICO 1: TRAIN vs VAL LOSS (Il più importante) ---
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'b-o', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'r-o', label='Validation Loss')
    plt.title('Total Loss: Train vs Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # --- GRAFICO 2: DETTAGLIO COMPONENTI (Mask vs Box) ---
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_mask_loss'], 'g--', label='Mask Loss (Forma)')
    plt.plot(epochs, history['train_box_loss'], 'm--', label='Box Loss (Posizione)')
    plt.title('Dettaglio Training: Mask vs Box')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Esegui la funzione
plot_metrics(history)

# TEST #

Funzione di test (inferenza del modello)

def predict_large_image(model, img_path, device, chip_size=512, stride=400, score_threshold=0.5):

    # 1. Carica l'immagine gigante
    img_full = cv2.imread(img_path)
    img_full = cv2.cvtColor(img_full, cv2.COLOR_BGR2RGB)
    h, w, _ = img_full.shape
    
    # Liste per salvare tutte le predizioni parziali
    all_boxes = []
    all_scores = []
    all_labels = []
    
    model.eval() # IMPORTANTE: mette il modello in modalità valutazione
    
    # 2. Sliding Window Loop
    print(f"Processando immagine {w}x{h}...")
    
    with torch.no_grad(): # Niente gradienti per risparmiare memoria
        for y in range(0, h, STRIDE):
            for x in range(0, w, STRIDE):
                # Coordinate del chip
                x2 = min(x + TILE_SIZE, w)
                y2 = min(y + TILE_SIZE, h)
                x1 = x2 - TILE_SIZE
                y1 = y2 - TILE_SIZE
                
                # Fix bordi negativi
                if x1 < 0: x1 = 0
                if y1 < 0: y1 = 0
                
                # Estrai il chip
                tile = img_full[y1:y2, x1:x2]
                
                # Prepara per il modello (ToTensor + Batch dimension)
                img_tensor = F.to_tensor(tile).to(device)
                img_tensor = img_tensor.unsqueeze(0) # Diventa [1, C, H, W]
                
                # --- PREDIZIONE ---
                prediction = model(img_tensor)[0]
                
                # Filtra per score (es. tieni solo se sicurezza > 50%)
                mask_score = prediction['scores'] > score_threshold
                boxes = prediction['boxes'][mask_score].cpu().numpy()
                scores = prediction['scores'][mask_score].cpu().numpy()
                labels = prediction['labels'][mask_score].cpu().numpy()
                
                # --- TRASFORMAZIONE COORDINATE (Da Locali a Globali) ---
                # Aggiungiamo l'offset (x1, y1) a ogni box
                for i in range(len(boxes)):
                    boxes[i][0] += x1 # xmin
                    boxes[i][1] += y1 # ymin
                    boxes[i][2] += x1 # xmax
                    boxes[i][3] += y1 # ymax
                    
                    all_boxes.append(boxes[i])
                    all_scores.append(scores[i])
                    all_labels.append(labels[i])

    # Se non ha trovato nulla, ritorna vuoto
    if len(all_boxes) == 0:
        return np.array([]), np.array([]), np.array([])

    # 3. GLOBAL NMS (Pulizia duplicati)
    # Convertiamo tutto in tensori per usare la funzione NMS di PyTorch
    all_boxes_t = torch.tensor(np.array(all_boxes)).float().to(device)
    all_scores_t = torch.tensor(np.array(all_scores)).float().to(device)
    all_labels_t = torch.tensor(np.array(all_labels)).int().to(device)
    
    # Applica NMS (iou_threshold=0.3 significa: se due box si sovrappongono per più del 30%, cancella quello più debole)
    keep_indices = torchvision.ops.nms(all_boxes_t, all_scores_t, iou_threshold=0.3)
    
    # Risultato finale pulito
    final_boxes = all_boxes_t[keep_indices].cpu().numpy()
    final_scores = all_scores_t[keep_indices].cpu().numpy()
    final_labels = all_labels_t[keep_indices].cpu().numpy()
    
    return final_boxes, final_scores, final_labels

# Esempio di come usare il Test Set alla fine
with open('/kaggle/working/test_annotations.json') as f:
    test_images_names = json.load(f)

# Prendi la prima immagine di test
test_img_name = test_images_names[0]
full_path = f"/kaggle/input/xview-dataset/train_images/train_images/{test_img_name}"

# Lancia la predizione sull'immagine intera
boxes, scores, labels = predict_large_image(model, full_path, device, ...)





# 1. Carica i pesi migliori salvati durante il training
# model = get_maskrcnn_model(num_classes=61) # Ricrea l'architettura
# model.load_state_dict(torch.load("model_epoch_2.pth")) # Carica i pesi
# model.to('cuda')

# 2. Scegli un'immagine di test
test_image_path = '/kaggle/input/xview-dataset/train_images/train_images/1045.tif' # O una cartella di test vera

# 3. Esegui la predizione
pred_boxes, pred_scores, pred_labels = predict_large_image(
    model, 
    test_image_path, 
    device='cuda',
    chip_size=512,
    stride=400,    # Stride < Chip_size per l'overlap!
    score_threshold=0.5
)

print(f"Trovati {len(pred_boxes)} oggetti nell'immagine intera!")

# 4. Visualizzazione (Plot)
# Carichiamo l'immagine per disegnarci sopra
img = cv2.imread(test_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, figsize=(12, 12))
ax.imshow(img)

# Disegna ogni box
for box in pred_boxes:
    xmin, ymin, xmax, ymax = box
    w_box = xmax - xmin
    h_box = ymax - ymin
    
    # Crea rettangolo rosso
    rect = patches.Rectangle(
        (xmin, ymin), w_box, h_box, 
        linewidth=2, edgecolor='r', facecolor='none'
    )
    ax.add_patch(rect)

plt.show()